In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import keras_tuner
from dataclasses import dataclass
from typing import List
from keras import layers, ops
from keras.utils import timeseries_dataset_from_array
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

from processing import split_indices, make_stationary, inverse_stationary, scale, unscale, sequence, Stationarizer, Normalizer
from tuner import LSTMLayerHyperparams, ModelHyperparams
from plotting import visualize_loss, show_plot
from model_builder import get_model, build_model, get_tuner

2024-08-02 16:32:59.507603: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data = pd.read_csv("data/YF_AAPL.csv")["Adj Close"].to_numpy().reshape(-1, 1)

train_idx = int(0.7 * np.shape(data)[0])
val_idx = int(0.85 * np.shape(data)[0])

train_data = data[:train_idx]
val_data = data[train_idx:val_idx]
test_data = data[val_idx:]

In [3]:
#train_stater = Stationarizer()
#val_stater = Stationarizer()
#normer = Normalizer()
normer = MinMaxScaler(feature_range=(0, 1), clip=True)
#train_data = train_stater.fit_transform(train_data)
#val_data = val_stater.fit_transform(val_data)
train_data = normer.fit_transform(train_data)
val_data = normer.transform(val_data)

In [4]:
train_ds = timeseries_dataset_from_array(
    train_data[:-20],
    train_data[20:],
    sequence_length=20,
    batch_size=8,
    shuffle=True
)
val_ds = timeseries_dataset_from_array(
    val_data[:-20],
    val_data[20:],
    sequence_length=20,
    batch_size=8,
    shuffle=False
)

for batch in train_ds.take(1):
    inputs, targets = batch

print("Input shape:", inputs.numpy().shape)
print("Target shape:", targets.numpy().shape)

Input shape: (8, 20, 1)
Target shape: (8, 1)


2024-08-02 16:33:02.245386: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [ ]:
tuner = get_tuner()
tuner.search(train_ds, epochs=20, validation_data=val_ds)


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
2                 |2                 |num_layers
112               |112               |units_0
tanh              |tanh              |activation_0
relu              |relu              |recurrent_activation_0
0                 |0                 |dropout_0
0.2               |0.2               |recurrent_dropout_0
relu              |relu              |activation_1
0.0022156         |0.0022156         |lr

Epoch 1/20
535/535 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - loss: 0.0319 - mse: 0.0319 - val_loss: 0.0046 - val_mse: 0.0046
Epoch 2/20
535/535 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 4.9582e-04 - mse: 4.9582e-04 - val_loss: 0.0016 - val_mse: 0.0016
Epoch 3/20
535/535 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 3.6154e-04 - mse: 3.6154e-04 - val_loss: 0.0038 - val_mse: 0.0038
Epoch 4/20
535/535 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 3.0053e-04 - mse: 3.0053e-04 - val_loss: 0.0025 - val_mse: 0.0025
Epoch 5/20
535/53